# Cross-filtering

[![open_in_colab][colab_badge]][colab_notebook_link]
[![open_in_binder][binder_badge]][binder_notebook_link]

[colab_badge]: https://colab.research.google.com/assets/colab-badge.svg
[colab_notebook_link]: https://colab.research.google.com/github/UnfoldedInc/examples/blob/master/notebooks/07%20-%20Crossfilter.ipynb
[binder_badge]: https://mybinder.org/badge_logo.svg
[binder_notebook_link]: https://mybinder.org/v2/gh/UnfoldedInc/examples/master?urlpath=lab/tree/notebooks/07%20-%20Crossfilter.ipynb

Cross-filtering is a technique often used in dashboards where filters are synced across multiple views which represent different facets of the same data. This example shows how Unfolded map filters can be coordinated with Plotly charts.

Let's create a local map and add data to it:

In [ ]:
from unfolded.map_sdk import UnfoldedMap
unfolded_map = UnfoldedMap()

In [ ]:
from sidecar import Sidecar
sc = Sidecar(title='Unfolded map', anchor='split-right')
with sc:
    display(unfolded_map)

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/UnfoldedInc/examples/master/notebooks/data/earthquakes.csv'
df = pd.read_csv(url)

In [ ]:
unfolded_map.add_dataset({
    'label': 'Earthquakes',
    'data': df
})

## Syncing a map filter with a Plotly histogram

We can use Plotly to draw a histogram showing the distribution of the numbers of earthquakes by their magnitude:

In [ ]:
import plotly.graph_objects as go

hist_fig = go.FigureWidget([go.Histogram(x = df['Magnitude'], nbinsx = 50)])
hist = hist_fig.data[0]

Let's now add event handlers. We supply an `on_selection` function that will be called when the selection in the Plotly histogram is changed:

In [ ]:
def on_histogram_selection_change(trace, points, state):
    # Update the Magnitude filter in the map
    unfolded_map.set_filter({
        'id': 'magnitude_filter',
        'field': 'Magnitude',
        'value': [min(points.xs), max(points.xs)]
    })
hist_fig.data[0].on_selection(on_histogram_selection_change)

The `on_selection` event handler will be called when a filter in the Unfolded map is changed (refer to the [Map SDK docs](https://docs.unfolded.ai/map-sdk/api/set-map-event-handlers) for more info on map event handling):

In [ ]:
def on_map_filter_change(event_info):
    # Update the selection in the histogram
    if 'Magnitude' in event_info['name'] and event_info['prop'] == 'value':
        v = event_info['value']
        condition = df['Magnitude'].between(v[0], v[1])
        indices = tuple(df.index[condition])
        hist.selectedpoints = indices

unfolded_map.set_map_event_handlers({
    'on_filter': on_map_filter_change
})

Now let's render the histogram:

In [ ]:
hist_fig.update_layout(
    title = 'Earthquakes by magnitude',
    height = 350, 
    xaxis = {'title': 'Magnitude'},
    yaxis = {'title': 'Count'},
    dragmode = 'select',
    hovermode = 'closest'
)
hist_fig

You can now open the left side bar in the map and select **Filters** in the top nav menu:

<img src=images/studio-filters-pane-2.jpg width=350>

Try changing the selection in the Plotly histogram by dragging (make sure you are using the **Box Select** tool). You should see the Magnitude filter update in the map when you change the selection in the Plotly histogram and vice versa:

<img src=images/crossfilter-480.gif width=480>

## Scatterplot

Let's now do the same with a more sophisticated chart, a scatterplot:

In [ ]:
import plotly.graph_objects as go
scatter_fig = go.FigureWidget([
    go.Scattergl(
        x = df['Magnitude'],
        y = df['Depth'], 
        marker = {'color': df['Depth'], 'size': df['Magnitude']}, 
        mode = 'markers'
    )
])
scatter = scatter_fig.data[0]

Let's again add event handlers. This function will be called when the selection in the scatterplot is changed:

In [ ]:
depth_filter = [df['Depth'].min(), df['Depth'].max()]
magnitude_filter = [df['Magnitude'].min(), df['Magnitude'].max()]

def on_scatterplot_selection_change(trace, points, state):
    # These need to be declared as global so that they
    # refer to the global scope variables defined above
    global magnitude_filter
    global depth_filter
    magnitude_filter = [
        min(points.xs, default = df['Magnitude'].min()), 
        max(points.xs, default = df['Magnitude'].max())
    ]
    depth_filter = [
        min(points.ys, default = df['Depth'].min()),
        max(points.ys, default = df['Depth'].max())
    ]
    unfolded_map.set_filter({
        'id': 'magnitude_filter',
        'field': 'Magnitude',
        'value': magnitude_filter 
    })
    unfolded_map.set_filter({
        'id': 'depth_filter',
        'field': 'Depth',
        'value': depth_filter
    })
scatter_fig.data[0].on_selection(on_scatterplot_selection_change)

Note that we are now setting two filters: one for `Magnitude`, another for `Depth`.

Let's add a map filter event handler:

In [ ]:
def on_map_filter_change2(event_info):
    global magnitude_filter
    global depth_filter
    if event_info['prop'] == 'value':
        if 'Magnitude' in event_info['name']:
            v = event_info['value']
            magnitude_filter = v
            condition = df[
                df['Magnitude'].between(v[0], v[1]) & 
                df['Depth'].between(depth_filter[0], depth_filter[1])
            ]
            scatter.selectedpoints = tuple(condition.index)
        elif 'Depth' in event_info['name']:
            v = event_info['value']
            depth_filter = v
            condition = df[
                df['Magnitude'].between(magnitude_filter[0], magnitude_filter[1]) & 
                df['Depth'].between(v[0], v[1])
            ]
            scatter.selectedpoints = tuple(condition.index)

# This will overwrite the previously set map on_filter event handler
unfolded_map.set_map_event_handlers({
    'on_filter': on_map_filter_change2
})

Let's now render the scatterplot:

In [ ]:
scatter_fig.update_layout(
    title = 'Earthquakes by depth and magnitude',
    width = 700, 
    height = 500, 
    xaxis = {'title': 'Magnitude'},
    yaxis = {'title': 'Depth'},
    dragmode = 'select',
    hovermode = 'closest'
)
scatter_fig

You can interactively select a subset of the data in the Plotly scatterplot (use the **Box Select** tool in the  Scatterplot chart toolbar, **Lasso Select** doesn't sync correctly yet). You should see the Magnitude and Depth filters automatically update in the map when you change the selection in the scatterplot.

Likewise, if you change one of the filters in the map, you should see the selection update in the Plotly scatterplot.